In [4]:
# Package Imports
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil


In [5]:
# Helper and script imports
import sys
sys.path.append("/Users/emilieyu/endotehelial-masboss")
from boolean_models.scripts.run_single import main
from boolean_models.analysis import (
    generate_ko_models,
    compute_delta,
    classify_phenotype,
)

In [6]:
# Set project root dir
PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
MODELS_DIR = PROJECT_ROOT / "boolean_models" / "models"
RESULTS_DIR = PROJECT_ROOT / "results" / "boolean_models"

# load model and WT configuration
model_file = MODELS_DIR / "rho.bnd"
cfg_file   = MODELS_DIR / "rho_base.cfg"

model = maboss.load(str(model_file), str(cfg_file))
nodes = ['DSP', 'TJP1', 'JCAD']

### Basic Perturbation Behaviour Validation

Generate perturbation scenarios, run perturbations, and store raw output

In [7]:
ko_scenarios = generate_ko_models(model, nodes)

In [8]:
def save_sim_details(result, result_directory, perb_name):
    # Save node probabilities
    nodes_probtraj = result.get_nodes_probtraj().rename_axis('t').reset_index()    
    nodes_probtraj.to_csv(
        result_directory / f"{perb_name}_nodes_probtraj.csv",
        index=False
    )

    # Save state probabilities
    states_probtraj = result.get_states_probtraj().rename_axis('t').reset_index()    
    states_probtraj.to_csv(
        result_directory / f"{perb_name}_states_probtraj.csv",
        index=False
    )


In [12]:
perb_dict = {}
phenotype_dict = {}

# Run simulations. 
for name, model in ko_scenarios.items():
    print(f"DEBUG: Running scenario: {name}")

    res_dir = RESULTS_DIR / "rho_model" / name
    
    # Ensure clean output directory
    if res_dir.exists():
        shutil.rmtree(res_dir)
    res_dir.mkdir(parents=True, exist_ok=True)

    # Run MaBoSS
    res = model.run()

    prob_df = res.get_nodes_probtraj()

    # Save raw simulation details
    save_sim_details(res, res_dir, name)
    
    # Compute Rho balance
    balance_df = prob_df.copy().rename_axis('t').reset_index()   
    balance_df["delta"] = compute_delta(balance_df)
    perb_dict[name] = balance_df

    # Compute raw phenotypes
    phenotype_df = classify_phenotype(balance_df).rename_axis('t').reset_index()   
    phenotype_dict[name] = phenotype_df

    # Compute raw phenotypes
    print(f"DEBUG: Writing intermediate data to file for: {name}")
    balance_df.to_csv(res_dir / f"{name}_phenotype.csv")
    phenotype_df.to_csv(res_dir / f"{name}_phenotype.csv")


DEBUG: Running scenario: WT
DEBUG: Writing intermediate data to file for: WT
DEBUG: Running scenario: DSP_ko
DEBUG: Writing intermediate data to file for: DSP_ko
DEBUG: Running scenario: TJP1_ko
DEBUG: Writing intermediate data to file for: TJP1_ko
DEBUG: Running scenario: JCAD_ko
DEBUG: Writing intermediate data to file for: JCAD_ko
DEBUG: Running scenario: DSP_TJP1_ko
DEBUG: Writing intermediate data to file for: DSP_TJP1_ko
DEBUG: Running scenario: DSP_JCAD_ko
DEBUG: Writing intermediate data to file for: DSP_JCAD_ko
DEBUG: Running scenario: TJP1_JCAD_ko
DEBUG: Writing intermediate data to file for: TJP1_JCAD_ko
DEBUG: Running scenario: DSP_TJP1_JCAD_ko
DEBUG: Writing intermediate data to file for: DSP_TJP1_JCAD_ko


In [15]:
tjp1 = perb_dict['TJP1_ko']
tjp1

,t,DSP,TJP1,JCAD,RhoA,RhoC,delta
0,0.0,0.494499,0.0,0.502700,0.540813,0.482015,-0.058798
1,0.1,0.494500,0.0,0.502700,0.582336,0.457086,-0.125250
2,0.2,0.494500,0.0,0.502700,0.604272,0.441025,-0.163247
3,0.3,0.494499,0.0,0.502700,0.618445,0.431285,-0.187160
4,0.4,0.494502,0.0,0.502701,0.631145,0.420570,-0.210575
5,0.5,0.494500,0.0,0.502699,0.635770,0.416423,-0.219347
6,0.6,0.494499,0.0,0.502699,0.641917,0.417531,-0.224386
7,0.7,0.494501,0.0,0.502700,0.647124,0.416122,-0.231002
8,0.8,0.494500,0.0,0.502700,0.653147,0.410364,-0.242783
9,0.9,0.494500,0.0,0.502699,0.657581,0.405227,-0.252354


In [17]:
pd.concat(
    [df.tail(5).assign(source=name) for name, df in perb_dict.items()],
    ignore_index=True
)

,t,DSP,TJP1,JCAD,RhoA,RhoC,delta,source
0,3.5,0.499002,0.500801,0.488203,0.590856,0.598647,0.007791,WT
1,3.6,0.499001,0.500800,0.488202,0.586501,0.598987,0.012486,WT
2,3.7,0.499001,0.500800,0.488199,0.588427,0.598584,0.010157,WT
3,3.8,0.499000,0.500801,0.488201,0.589567,0.601648,0.012081,WT
4,3.9,0.499000,0.500802,0.488199,0.590854,0.603403,0.012549,WT
5,3.5,0.000000,0.504802,0.500002,0.399223,0.677261,0.278038,DSP_ko
6,3.6,0.000000,0.504801,0.500001,0.398695,0.680870,0.282175,DSP_ko
7,3.7,0.000000,0.504800,0.500000,0.401870,0.677318,0.275448,DSP_ko
8,3.8,0.000000,0.504800,0.499999,0.403962,0.676962,0.273000,DSP_ko
9,3.9,0.000000,0.504799,0.500000,0.403589,0.678397,0.274808,DSP_ko


### Plotting Results